# Load the data from personal Drive

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
path_to_file = '/content/drive/MyDrive/prediction_ready_final_dataset.parquet'
genes_expression = pd.read_parquet(path_to_file)
genes_expression.head()

Mounted at /content/drive


,sig_id,pert_id,pert_iname,pert_type,cell_id,pert_idose,pert_itime,distil_id,embedding,780,...,7264,5467,2767,23038,57048,79716,pert_itime_value,pert_itime_unit,pert_idose_value,pert_idose_unit
0,LJP005_A375_24H:A03,DMSO,DMSO,ctl_vehicle,A375,None,24 h,LJP005_A375_24H_X1_B19:A03|LJP005_A375_24H_X2_...,"[0.5616036, -0.13782915, 0.2569403, 0.05337388...",-0.154526,...,0.323543,0.198538,-0.075226,-1.076814,-0.240803,0.105998,24,h,NaN,None
1,LJP005_A375_24H:A04,DMSO,DMSO,ctl_vehicle,A375,None,24 h,LJP005_A375_24H_X1_B19:A04|LJP005_A375_24H_X2_...,"[0.5616036, -0.13782915, 0.2569403, 0.05337388...",0.113874,...,0.197536,-0.177739,0.063853,-0.904253,0.040530,-0.619656,24,h,NaN,None
2,LJP005_A375_24H:A05,DMSO,DMSO,ctl_vehicle,A375,None,24 h,LJP005_A375_24H_X1_B19:A05|LJP005_A375_24H_X2_...,"[0.5616036, -0.13782915, 0.2569403, 0.05337388...",-0.038252,...,-0.216541,0.887142,0.156324,-0.939757,-0.093534,-0.260954,24,h,NaN,None
3,LJP005_A375_24H:A06,DMSO,DMSO,ctl_vehicle,A375,None,24 h,LJP005_A375_24H_X1_B19:A06|LJP005_A375_24H_X2_...,"[0.5616036, -0.13782915, 0.2569403, 0.05337388...",0.466993,...,-0.354371,0.903572,0.505928,-0.468805,1.793977,-0.642835,24,h,NaN,None
4,LJP005_A375_24H:A07,BRD-K76908866,CP-724714,trt_cp,A375,10.0 um,24 h,LJP005_A375_24H_X1_B19:A07|LJP005_A375_24H_X2_...,"[0.29808575, 0.10792902, -0.66687685, 0.768152...",-1.462477,...,-6.896286,0.728512,-0.183455,0.852644,1.406259,-0.416583,24,h,10.0,um


# Here we defined the target values as the ids of the gene's expression difference. For the features we kept the ones related to the drug such as the embedding and the rest. We dropped sig_id , pert_id and distil_id because they contain unique identifiers rather than actual features that could be useful for our random forest model.

In [ ]:
numeric_cols = genes_expression.select_dtypes(include=['number']).columns
gene_expression_cols = [col for col in numeric_cols if str(col).isdigit()]


feature_cols = ['pert_idose', 'pert_itime', 'pert_iname', 'pert_type', 'cell_id', 'embedding','pert_idose_unit', 'pert_itime_unit']

id_cols_to_drop = ['sig_id', 'pert_id', 'distil_id']

X = genes_expression[feature_cols].copy()
X = X.drop(columns=id_cols_to_drop, errors='ignore')
Y = genes_expression[gene_expression_cols].copy()

print("Shape of X (features):", X.shape)
print("Shape of Y (targets):", Y.shape)
print("X head:")
print(X.head())
print("Y head:")
print(Y.head())

Shape of X (features): (111329, 8)
Shape of Y (targets): (111329, 978)
X head:
  pert_idose pert_itime pert_iname    pert_type cell_id  \
0       None       24 h       DMSO  ctl_vehicle    A375   
1       None       24 h       DMSO  ctl_vehicle    A375   
2       None       24 h       DMSO  ctl_vehicle    A375   
3       None       24 h       DMSO  ctl_vehicle    A375   
4    10.0 um       24 h  CP-724714       trt_cp    A375   

                                           embedding pert_idose_unit  \
0  [0.5616036, -0.13782915, 0.2569403, 0.05337388...            None   
1  [0.5616036, -0.13782915, 0.2569403, 0.05337388...            None   
2  [0.5616036, -0.13782915, 0.2569403, 0.05337388...            None   
3  [0.5616036, -0.13782915, 0.2569403, 0.05337388...            None   
4  [0.29808575, 0.10792902, -0.66687685, 0.768152...              um   

  pert_itime_unit  
0               h  
1               h  
2               h  
3               h  
4               h  
Y head:
     

# Preprocess 'embedding' column. We preprocessed the embedding as most ML models (including RF) work with individual numerical values, not lists of ones (our embedding is contained in a list of values). By expanding the list into distinct numerical values, the data is more usable by the model for training.


In [ ]:
embedding_df = X['embedding'].apply(pd.Series)
embedding_df = embedding_df.add_prefix('embedding_')
X = pd.concat([X, embedding_df], axis=1)
X = X.drop('embedding', axis=1)

print("Shape of X after expanding embeddings:", X.shape)
print("X head after embedding expansion:")
print(X.head())

Shape of X after expanding embeddings: (111329, 391)
X head after embedding expansion:
  pert_idose pert_itime pert_iname    pert_type cell_id pert_idose_unit  \
0       None       24 h       DMSO  ctl_vehicle    A375            None   
1       None       24 h       DMSO  ctl_vehicle    A375            None   
2       None       24 h       DMSO  ctl_vehicle    A375            None   
3       None       24 h       DMSO  ctl_vehicle    A375            None   
4    10.0 um       24 h  CP-724714       trt_cp    A375              um   

  pert_itime_unit  embedding_0  embedding_1  embedding_2  ...  embedding_374  \
0               h     0.561604    -0.137829     0.256940  ...      -0.149551   
1               h     0.561604    -0.137829     0.256940  ...      -0.149551   
2               h     0.561604    -0.137829     0.256940  ...      -0.149551   
3               h     0.561604    -0.137829     0.256940  ...      -0.149551   
4               h     0.298086     0.107929    -0.666877  ... 

# We performed a one-hot encoding on our features besides the embedding because our RF requires numerical input to perform better.

In [ ]:
import re

def extract_value_unit(s):
    if pd.isna(s) or str(s).lower() == 'none':
        return None, None
    s = str(s).strip().replace(',', '.')
    match = re.match(r'([0-9.]+)\s*(.*)', s)
    if match:
        value = float(match.group(1))
        unit = match.group(2).strip()
        return value, unit if unit else None
    return None, None


X['pert_idose_value'], X['pert_idose_unit'] = zip(*X['pert_idose'].apply(extract_value_unit))
X['pert_idose_value'] = pd.to_numeric(X['pert_idose_value'], errors='coerce')
X = X.drop('pert_idose', axis=1)


X['pert_itime_value'], X['pert_itime_unit'] = zip(*X['pert_itime'].apply(extract_value_unit))
X['pert_itime_value'] = pd.to_numeric(X['pert_itime_value'], errors='coerce')
X = X.drop('pert_itime', axis=1)


categorical_cols = ['pert_iname', 'pert_type', 'cell_id', 'pert_idose_unit', 'pert_itime_unit']

for col in categorical_cols:
    X[col] = X[col].fillna('NaN').astype(str)
X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

print("Shape of X after one-hot encoding categorical features:", X.shape)
print("X head after one-hot encoding:")
print(X.head())

Shape of X after one-hot encoding categorical features: (111329, 2169)
X head after one-hot encoding:
   embedding_0  embedding_1  embedding_2  embedding_3  embedding_4  \
0     0.561604    -0.137829     0.256940     0.053374     0.417494   
1     0.561604    -0.137829     0.256940     0.053374     0.417494   
2     0.561604    -0.137829     0.256940     0.053374     0.417494   
3     0.561604    -0.137829     0.256940     0.053374     0.417494   
4     0.298086     0.107929    -0.666877     0.768152     0.126540   

   embedding_5  embedding_6  embedding_7  embedding_8  embedding_9  ...  \
0     0.797337    -1.176480     0.803319     0.029956     0.219498  ...   
1     0.797337    -1.176480     0.803319     0.029956     0.219498  ...   
2     0.797337    -1.176480     0.803319     0.029956     0.219498  ...   
3     0.797337    -1.176480     0.803319     0.029956     0.219498  ...   
4    -0.001346    -0.387187     0.505446    -0.111360     0.147754  ...   

   cell_id_NPC.TAK  cell_i

# Split data : Split data: Because the dataset was too big to perform a random forest on its entirety, we did it on only 10% of it (approximately 10,000 lines). We also chose an 80-20% training/test splitting ratio.


In [ ]:
from sklearn.model_selection import train_test_split

# Sample a fraction of the data to reduce the dataset size for faster training
sample_fraction = 0.1 # You can adjust this value (e.g., 0.01 for very fast, 0.5 for more data)
X_sample, _, Y_sample, _ = train_test_split(X, Y, test_size=1-sample_fraction, random_state=42)


X_train, X_test, Y_train, Y_test = train_test_split(X_sample, Y_sample, test_size=0.2, random_state=42)


print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of Y_train:", Y_train.shape)
print("Shape of Y_test:", Y_test.shape)

Shape of X_train: (8905, 2169)
Shape of X_test: (2227, 2169)
Shape of Y_train: (8905, 978)
Shape of Y_test: (2227, 978)


# Train RandomForestRegressor :  We choose to build 10 trees. When trying to add more the Google Colab was not executing the cell due to a lack of RAM.




In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=10, random_state=42, n_jobs=-1)

# Train the model
print("Training RandomForestRegressor model...")
model.fit(X_train, Y_train)
print("Model training complete.")

Training RandomForestRegressor model...
Model training complete.


# Evaluate Model Performance
We computed the accuracy of the model in two ways.
Firstly, we computed the MAE and R squared score for the global model. These scores are averaged across all gene 'model' performances. Secondly, we ranked the individual gene's R-squared from best to worst to understand the extent to which the model was able to predict certain genes over others.

In [16]:
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score

Y_pred = model.predict(X_test)

mae_scores = []
r2_scores = []

for i in range(Y_test.shape[1]):
    mae_scores.append(mean_absolute_error(Y_test.iloc[:, i], Y_pred[:, i]))
    r2_scores.append(r2_score(Y_test.iloc[:, i], Y_pred[:, i]))


average_mae = np.mean(mae_scores)
average_r2 = np.mean(r2_scores)

print(f"Average Mean Absolute Error (MAE) across all gene targets: {average_mae:.4f}")
print(f"Average R-squared (R2) score across all gene targets: {average_r2:.4f}")

Average Mean Absolute Error (MAE) across all gene targets: 0.8701
Average R-squared (R2) score across all gene targets: 0.1228


In [17]:
import pandas as pd

# Create a DataFrame to store gene and their R2 scores
r2_df = pd.DataFrame({'Gene': Y.columns, 'R2_Score': r2_scores})

# Sort the DataFrame by R2_Score in descending order
r2_df_sorted = r2_df.sort_values(by='R2_Score', ascending=False)

print("Top 10 best predicted genes by R2 score:")
print(r2_df_sorted.head(10))

print("\nTop 10 worst predicted genes by R2 score:")
print(r2_df_sorted.tail(10))

Top 10 best predicted genes by R2 score:
      Gene  R2_Score
17    3337  0.615787
82    7153  0.599553
318  11065  0.595221
348  11041  0.576689
248   6284  0.564309
156   3315  0.562479
353    983  0.560736
36    5708  0.552018
825  10652  0.545910
243   2582  0.544681

Top 10 worst predicted genes by R2 score:
      Gene  R2_Score
283  23224 -0.139493
514   8800 -0.141776
958  23410 -0.144602
354   8869 -0.148776
46    2778 -0.149782
935   6676 -0.155986
402    178 -0.156556
241   6616 -0.167315
533    695 -0.167515
919  80204 -0.184738


The output above shows the top 10 best and worst predicted genes based on their R-squared scores. A higher R2 score indicates a better fit of the model to that specific gene's expression data.

# Cross-validation using K-fold. Before concluding on the results, and due to the small number of trees, we decided to cross-validate our results using the statistical method K-fold. We split the dataset into 5 equal parts and then iteratively considered one of the parts as testing and the rest as training, comparing the results.

## Global performance over each fold.

In [18]:
from sklearn.model_selection import KFold

# Initialize KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Print the total number of samples for cross-validation
print(f"Total number of samples for cross-validation: {X_sample.shape[0]}")

Total number of samples for cross-validation: 11132


In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np

# Initialize lists to store metrics for each fold
fold_mae_scores = []
fold_r2_scores = []

# Initialize lists to store metrics for each gene across folds
gene_mae_scores_per_fold = []
gene_r2_scores_per_fold = []

# Loop through each fold
for fold, (train_index, val_index) in enumerate(kf.split(X_sample)):
    print(f"\n--- Fold {fold+1}/{kf.n_splits} ---")

    # Split data into training and validation sets for the current fold
    X_train_fold, X_val_fold = X_sample.iloc[train_index], X_sample.iloc[val_index]
    Y_train_fold, Y_val_fold = Y_sample.iloc[train_index], Y_sample.iloc[val_index]

    # Initialize and train the RandomForestRegressor model
    model_fold = RandomForestRegressor(n_estimators=10, random_state=42, n_jobs=-1)
    print(f"Training model for Fold {fold+1}...")
    model_fold.fit(X_train_fold, Y_train_fold)
    print(f"Model training complete for Fold {fold+1}.")

    # Make predictions on the validation set
    Y_pred_fold = model_fold.predict(X_val_fold)

    # Calculate MAE and R2 for each gene target in the current fold
    current_fold_gene_mae = []
    current_fold_gene_r2 = []

    for i in range(Y_val_fold.shape[1]):
        # Ensure there's enough data for calculation
        if len(Y_val_fold.iloc[:, i].dropna()) > 0 and len(Y_pred_fold[:, i]) > 0:
            current_fold_gene_mae.append(mean_absolute_error(Y_val_fold.iloc[:, i], Y_pred_fold[:, i]))
            current_fold_gene_r2.append(r2_score(Y_val_fold.iloc[:, i], Y_pred_fold[:, i]))
        else:
            current_fold_gene_mae.append(np.nan) # Append NaN if no valid data
            current_fold_gene_r2.append(np.nan) # Append NaN if no valid data

    # Store individual gene scores for the fold
    gene_mae_scores_per_fold.append(current_fold_gene_mae)
    gene_r2_scores_per_fold.append(current_fold_gene_r2)

    # Calculate and store average MAE and R2 for the current fold
    avg_mae_fold = np.nanmean(current_fold_gene_mae) # Use nanmean to handle NaNs gracefully
    avg_r2_fold = np.nanmean(current_fold_gene_r2)

    fold_mae_scores.append(avg_mae_fold)
    fold_r2_scores.append(avg_r2_fold)

    print(f"Average MAE for Fold {fold+1}: {avg_mae_fold:.4f}")
    print(f"Average R2 for Fold {fold+1}: {avg_r2_fold:.4f}")

print("\nCross-validation complete.")
print(f"Overall Average MAE: {np.nanmean(fold_mae_scores):.4f}")
print(f"Overall Average R2: {np.nanmean(fold_r2_scores):.4f}")


--- Fold 1/5 ---
Training model for Fold 1...
Model training complete for Fold 1.
Average MAE for Fold 1: 0.8674
Average R2 for Fold 1: 0.1295

--- Fold 2/5 ---
Training model for Fold 2...
Model training complete for Fold 2.
Average MAE for Fold 2: 0.8840
Average R2 for Fold 2: 0.1389

--- Fold 3/5 ---
Training model for Fold 3...
Model training complete for Fold 3.
Average MAE for Fold 3: 0.8693
Average R2 for Fold 3: 0.1406

--- Fold 4/5 ---
Training model for Fold 4...
Model training complete for Fold 4.
Average MAE for Fold 4: 0.8783
Average R2 for Fold 4: 0.1588

--- Fold 5/5 ---
Training model for Fold 5...
Model training complete for Fold 5.
Average MAE for Fold 5: 0.8730
Average R2 for Fold 5: 0.1695

Cross-validation complete.
Overall Average MAE: 0.8744
Overall Average R2: 0.1475


In [20]:
import numpy as np
import pandas as pd

# 1. Calculate and print the mean and standard deviation of global MAE and R2 scores
mean_mae_global = np.nanmean(fold_mae_scores)
std_mae_global = np.nanstd(fold_mae_scores)
mean_r2_global = np.nanmean(fold_r2_scores)
std_r2_global = np.nanstd(fold_r2_scores)

print(f"Overall Cross-Validation Results (across {kf.n_splits} folds):")
print(f"  Mean MAE: {mean_mae_global:.4f} (Std: {std_mae_global:.4f})")
print(f"  Mean R2: {mean_r2_global:.4f} (Std: {std_r2_global:.4f})")

# 2. Convert gene-wise scores per fold into NumPy arrays
gene_mae_scores_array = np.array(gene_mae_scores_per_fold)
gene_r2_scores_array = np.array(gene_r2_scores_per_fold)

# 3. Calculate average and standard deviation of MAE and R2 for each gene across all folds
average_gene_mae = np.nanmean(gene_mae_scores_array, axis=0)
std_gene_mae = np.nanstd(gene_mae_scores_array, axis=0)
average_gene_r2 = np.nanmean(gene_r2_scores_array, axis=0)
std_gene_r2 = np.nanstd(gene_r2_scores_array, axis=0)

# 4. Create a Pandas DataFrame for gene-wise R2 scores (averaged across folds)
r2_gene_df = pd.DataFrame({
    'Gene': Y_sample.columns,
    'Average_R2': average_gene_r2,
    'Std_R2': std_gene_r2
})

# 5. Sort the DataFrame by 'Average_R2' in descending order
r2_gene_df_sorted = r2_gene_df.sort_values(by='Average_R2', ascending=False)

# 6. Print the top 10 and bottom 10 genes based on average R2
print("\nTop 10 best predicted genes by Average R2 score (across folds):")
print(r2_gene_df_sorted.head(10))

print("\nTop 10 worst predicted genes by Average R2 score (across folds):")
print(r2_gene_df_sorted.tail(10))

Overall Cross-Validation Results (across 5 folds):
  Mean MAE: 0.8744 (Std: 0.0061)
  Mean R2: 0.1475 (Std: 0.0145)

Top 10 best predicted genes by Average R2 score (across folds):
      Gene  Average_R2    Std_R2
17    3337    0.619294  0.009513
82    7153    0.606025  0.011567
243   2582    0.595013  0.022907
347   6275    0.592893  0.040783
248   6284    0.583869  0.016348
318  11065    0.583095  0.017256
348  11041    0.581578  0.021055
156   3315    0.581571  0.012510
274   2673    0.577960  0.021810
30    3303    0.575450  0.018844

Top 10 worst predicted genes by Average R2 score (across folds):
      Gene  Average_R2    Std_R2
935   6676   -0.120260  0.020163
387   6304   -0.120410  0.023205
528   9488   -0.120510  0.041905
839  51056   -0.126520  0.018781
668   2770   -0.126554  0.014853
467  10610   -0.126563  0.032959
958  23410   -0.130356  0.023715
551   5583   -0.139982  0.017455
776   6714   -0.150949  0.026788
919  80204   -0.185215  0.034604


## Ranking the best and worst performing genes over each fold.

In [22]:
# --- Display top/bottom 10 genes for each K-fold ---
print("\n--- Individual K-Fold Gene Predictions ---")
for fold_idx, r2_scores_fold in enumerate(gene_r2_scores_per_fold):
    print(f"\nFold {fold_idx + 1}:")
    fold_r2_df = pd.DataFrame({
        'Gene': Y_sample.columns,
        'R2_Score': r2_scores_fold
    })
    fold_r2_df_sorted = fold_r2_df.sort_values(by='R2_Score', ascending=False)

    print("  Top 10 best predicted genes by R2 score for this fold:")
    print(fold_r2_df_sorted.head(10))

    print("\n  Top 10 worst predicted genes by R2 score for this fold:")
    print(fold_r2_df_sorted.tail(10))


--- Individual K-Fold Gene Predictions ---

Fold 1:
  Top 10 best predicted genes by R2 score for this fold:
      Gene  R2_Score
17    3337  0.608319
82    7153  0.593214
156   3315  0.584133
318  11065  0.580379
348  11041  0.569062
248   6284  0.560484
36    5708  0.556794
243   2582  0.554177
380  26292  0.551162
285  30836  0.550669

  Top 10 worst predicted genes by R2 score for this fold:
      Gene  R2_Score
514   8800 -0.131950
241   6616 -0.131998
758  23142 -0.137226
387   6304 -0.144907
354   8869 -0.147831
935   6676 -0.149723
896  29763 -0.152780
626   6774 -0.156114
533    695 -0.182499
919  80204 -0.219546

Fold 2:
  Top 10 best predicted genes by R2 score for this fold:
      Gene  R2_Score
17    3337  0.624090
82    7153  0.619355
243   2582  0.604551
30    3303  0.603375
347   6275  0.599639
156   3315  0.588245
274   2673  0.582811
248   6284  0.580405
36    5708  0.575524
725  27032  0.569970

  Top 10 worst predicted genes by R2 score for this fold:
      Gene  R

#

# Result conclusion: On one hand, the RandomForestRegressor we fitted with 10 estimators has a low predictive power across all 978 genes, with an R-squared of 14.75% and a mean absolute error of 0.874. The model was fitted with a small number of trees and on 10% of the original dataset size due to a lack of computational power, even though we used an A100 GPU on Google Colab. On the other hand, the model's predictive power has high variance. As a matter of fact, it is able to achieve a moderate predictive power (R-squared above 0.5) on some specific genes, while also showing worse performance than the baseline on another set of genes. The results are consistent across all five folds of the K-fold, displaying the robustness of the results on the dataset.

# Here we downloaded the trained models using pickle such that we can reuse it  without having to train it again.

In [26]:
import pickle
from google.colab import files
import shutil
import os

model_filename = 'random_forest_model.pkl'
with open(model_filename, 'wb') as file:
    pickle.dump(model_fold, file)

print(f"Modèle enregistré sous {model_filename}")


files.download(model_filename)

drive_path = '/content/drive/MyDrive/random_forest_model.pkl'

try:
    shutil.copy(model_filename, drive_path)
    print(f"Le modèle '{model_filename}' a été copié avec succès sur Google Drive à : {drive_path}")
except FileNotFoundError:
    print(f"Erreur : Le fichier '{model_filename}' n'a pas été trouvé. Veuillez vous assurer qu'il a été enregistré localement.")
except Exception as e:
    print(f"Une erreur est survenue lors de la copie du fichier sur Google Drive : {e}")

Modèle enregistré sous random_forest_model.pkl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Le modèle 'random_forest_model.pkl' a été copié avec succès sur Google Drive à : /content/drive/MyDrive/random_forest_model.pkl
